In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import pandas as pd

df = pd.read_csv('permisos_construccion.csv')

### Tomar una decisión respecto a los datos faltantes, duplicados, inconsistencias y cualquier inconveniente que encuentren con los datos.

+ `Street Name` tiene distintos tipos de usos de mayúsculas y minúsculas. Se va a pasar todo a mayúsculas en la primera letra y el resto en minúsculas.
+ `Street Suffix` tiene valores nulos y valores que no son nulos. Se va a unir esta columna con `Street Name` y se va a eliminar `Street Name Suffix`.


In [ ]:
# `Street Name` tiene distintos tipos de usos de mayúsculas y minúsculas. Se va a pasar todo a mayúsculas en la primera letra y el resto en minúsculas.
df['Street Name'] = df['Street Name'].str.title() + ' ' + df['Street Suffix'].fillna('')
df.drop(columns=['Street Suffix'], inplace=True)
